<a href="https://colab.research.google.com/github/thaiphi/Stock-Markets-Analytics-2024/blob/main/Week1/Homework_1_Intro_and_Data_Souces_for_Stock_Markets_Analytics_Zoomcamp_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install main library YFinance
!pip install --upgrade yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

import duckdb

## Question 1 : Question 1: [Macro] Average growth of GDP in 2023
What is the average growth (in %) of GDP in 2023?

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer 5.7

In [ ]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=23
Period for indexes: 1954-04-23 to 2024-04-23 


In [ ]:
#Read https://fred.stlouisfed.org/series/GDPC1)
gdpc1 = pd.DataFrame()
gdpc1 = pdr.DataReader("gdpc1", "fred", start=start)
gdpc1.reset_index(inplace=True)
gdpc1.tail()

,DATE,gdpc1
273,2022-10-01,21989.981
274,2023-01-01,22112.329
275,2023-04-01,22225.350
276,2023-07-01,22490.692
277,2023-10-01,22679.255


In [ ]:

# Connect to DuckDB
con = duckdb.connect(database=':memory:', read_only=False)


# Write the Pandas DataFrame to DuckDB
con.register('gdpc1', gdpc1)

# Query the DuckDB table
result = con.execute('SELECT * FROM gdpc1' )
print(result.fetchdf())

          DATE      gdpc1
0   1954-07-01   2880.482
1   1954-10-01   2936.852
2   1955-01-01   3020.746
3   1955-04-01   3069.910
4   1955-07-01   3111.379
..         ...        ...
273 2022-10-01  21989.981
274 2023-01-01  22112.329
275 2023-04-01  22225.350
276 2023-07-01  22490.692
277 2023-10-01  22679.255

[278 rows x 2 columns]


https://www.investopedia.com/terms/y/year-over-year.asp

In [ ]:
# prompt: Query the DuckDB table for 2023

result = con.execute('''
SELECT * FROM gdpc1 WHERE
    date >= '2023-01-01' AND date <= '2023-12-31'
    ''')
print(result.fetchdf())



        DATE      gdpc1
0 2023-01-01  22112.329
1 2023-04-01  22225.350
2 2023-07-01  22490.692
3 2023-10-01  22679.255


In [ ]:
# Calculate the YoY growth rate
con.execute('''
    CREATE VIEW gdpc1_yoy AS
    SELECT
        DATE,
        GDPC1,
        LAG(GDPC1, 4) OVER (ORDER BY date) AS value_one_year_ago
    FROM
        gdpc1
''')

# Filter for the year 2023
con.execute('''
    CREATE VIEW gdpc1_yoy_2023 AS
    SELECT
        DATE,
        GDPC1,
        value_one_year_ago,
        ROUND((GDPC1 / value_one_year_ago - 1) * 100, 1) AS yoy_growth_rate
    FROM
        gdpc1_yoy
    WHERE
        date >= '2023-01-01' AND date <= '2023-12-31'
''')

# Calculate the average YoY growth rate for 2023
result = con.execute('SELECT AVG(yoy_growth_rate) FROM gdpc1_yoy_2023')
average_yoy_growth_2023 = result.fetchone()[0]

print(f'Average YoY growth rate in 2023: {average_yoy_growth_2023:.1f}%')

Average YoY growth rate in 2023: 2.5%


## Question 2. [Macro] Inverse "Treasury Yield"
Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.

Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2, https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

In [ ]:
# https://fred.stlouisfed.org/series/DGS2
# https://fred.stlouisfed.org/series/DGS10
DGS2 = pdr.DataReader("DGS2", "fred", start=start)
DGS10= pdr.DataReader("DGS10", "fred", start=start)
# Reset index to make 'DATE' a column for both DataFrames
DGS2.reset_index(inplace=True)
DGS10.reset_index(inplace=True)

In [ ]:
# Connect to DuckDB
con = duckdb.connect(database=':memory:', read_only=False)

# Write the Pandas DataFrame to DuckDB
con.register('DGS2', DGS2)
con.register('DGS10', DGS10)
# Join the DataFrames on the 'DATE' column
result = con.execute('''
  SELECT DGS2."DATE", DGS2."DGS2", DGS10."DGS10",
  DGS10."DGS10" - DGS2."DGS2" AS "DGS10_DGS2_Difference"
  FROM DGS2
  JOIN DGS10 ON DGS2."DATE" = DGS10."DATE"
  WHERE DGS2."DATE" >= '2000-01-01'
''')

# Fetch the result as a DataFrame
joined_df = result.fetchdf()

# Print the joined DataFrame
print(joined_df)

           DATE  DGS2  DGS10  DGS10_DGS2_Difference
0    2000-01-03  6.38   6.58                   0.20
1    2000-01-04  6.30   6.49                   0.19
2    2000-01-05  6.38   6.62                   0.24
3    2000-01-06  6.35   6.57                   0.22
4    2000-01-12  6.49   6.72                   0.23
...         ...   ...    ...                    ...
6336 2023-12-26  4.26   3.89                  -0.37
6337 2017-07-11  1.37   2.37                   1.00
6338 2020-06-05  0.22   0.91                   0.69
6339 2023-12-25   NaN    NaN                    NaN
6340 2024-02-13  4.64   4.31                  -0.33

[6341 rows x 4 columns]


In [ ]:
# prompt: Find the min value of (DGS10_DGS2_Difference) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point. ising duckdb

# Find the minimum value of DGS10_DGS2_Difference after 2000-01-01
result = con.execute('''
SELECT MIN(DGS10_DGS2_Difference) AS min_diff
FROM joined_df
WHERE "DATE" >= '2000-01-01'
''')

# Fetch the result as a DataFrame
min_diff_df = result.fetchdf()

# Print the minimum value
min_diff = min_diff_df['min_diff'][0]

# Round to 1 digit after the decimal point and print
print(f'Minimum value of DGS10_DGS2_Difference after 2000-01-01: {round(min_diff, 1)}')


Minimum value of DGS10_DGS2_Difference after 2000-01-01: -1.1


In [ ]:
# prompt: visualize DGS10_DGS2_Difference DGS10 and DGS2 also draw a horizontal line thorugh zero

# Create a new column for the zero line
joined_df['Zero Line'] = 0

# Create the plot
fig = go.Figure()

# Add traces for DGS10_DGS2_Difference, DGS10 and DGS2
fig.add_trace(go.Scatter(x=joined_df['DATE'], y=joined_df['DGS10_DGS2_Difference'], name='DGS10_DGS2_Difference'))
fig.add_trace(go.Scatter(x=joined_df['DATE'], y=joined_df['DGS10'], name='DGS10'))
fig.add_trace(go.Scatter(x=joined_df['DATE'], y=joined_df['DGS2'], name='DGS2'))

# Add a horizontal line for zero
fig.add_hline(y=0, line_color="black")

# Update the layout
fig.update_layout(
    title='DGS10_DGS2_Difference, DGS10 and DGS2 with Zero Line',
    xaxis_title='Date',
    yaxis_title='Value',
)

# Show the plot
fig.show()


(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)


https://www.reuters.com/markets/rates-bonds/us-treasury-key-yield-curve-inversion-becomes-longest-record-2024-03-21/
According to Reuters, the yield curve between 2-year and 10-year Treasury bonds has surpassed a record 659 days of inversion. This  is usually warning sign of a coming recession for two  reasons :

**Inflation and the Fed's Response**: Due to rising inflation, investors anticipate the Federal Reserve (Fed) to raise short-term interest rates aggressively to control inflation. The demand drives short-term bond yields higher than long-term ones.

**Impact on Borrowing and Investment**: An inverted yield curve can have negative consequences for the economy in two ways. First, higher short-term rates make borrowing more expensive in which discouraging investment and spending. Second, lower yields on long-term bonds make them less attractive to investors thus new ventures and projects.
However, the recession has not occur at least the definition of recesison has not occured yet due to two reason

1.   High Consumer Savings: The people have saves during or after the pandemic due to stimulus package, student loans freeze, Covis small bussiness relief, etc. This financial cushion allows them to continue spending even if borrowing costs rise due to the inverted yield curve.
2.   Fed's Actions: The Fed's role in containing "banking turmoil"  by offering emergency liquidity measures to ease credit conditions. By doing this, the Fed might help prevent banks from becoming overly cautious and restricting lending, which could further slow down the economy.

Regardless the inverted yeild curve still negatively impact the economic expansion:

1.   Positive Return for Risk: In a healthy economy, the yield curve slopes upwards.This means investors can earn a higher return for taking on the risk of lending money on long term bond. This incentivizes them to invest in long-term projects and businesses, which fuels economic growth.
2.   Inverted Curve Discourages Risk: When the yield curve inverts, short-term investments offer a higher return than long-term ones. This disincentivizes investors from taking on the additional risk of long-term investments. They might choose to park their money in safer, shorter-term options.







## Question 3. [Index] Which Index is better recently?
Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

In [ ]:
# Download historical data for S&P 500 (^GSPC) and IPC Mexico (^MXX)
gspc_data = pd.DataFrame()
mxx_data = pd.DataFrame()
gspc_data = yf.download('^GSPC', start='2019-04-09', end='2024-04-09')
mxx_data = yf.download('^MXX', start='2019-04-09', end='2024-04-09')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Calculate the 5-year growth for each index
gspc_start_price = gspc_data['Adj Close'].iloc[0]
gspc_end_price = gspc_data['Adj Close'].iloc[-1]
gspc_growth = ((gspc_end_price / gspc_start_price) - 1) * 100

mxx_start_price = mxx_data['Adj Close'].iloc[0]
mxx_end_price = mxx_data['Adj Close'].iloc[-1]
mxx_growth = ((mxx_end_price / mxx_start_price) - 1) * 100

# Determine the higher growing index and its growth rate
higher_growing_index = '^GSPC' if gspc_growth > mxx_growth else '^MXX'
higher_growth_rate = round(gspc_growth) if gspc_growth > mxx_growth else round(mxx_growth)

# Print the results
print(f"Higher growing index: {higher_growing_index}")
print(f"Growth rate: {higher_growth_rate}%")

Higher growing index: ^GSPC
Growth rate: 81%


In [ ]:
gspc_data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-04-02,5204.290039,5208.339844,5184.049805,5205.810059,5205.810059,3886590000
2024-04-03,5194.370117,5228.750000,5194.370117,5211.490234,5211.490234,3703250000
2024-04-04,5244.049805,5256.589844,5146.060059,5147.209961,5147.209961,4075680000
2024-04-05,5158.950195,5222.180176,5157.209961,5204.339844,5204.339844,3386780000
2024-04-08,5211.370117,5219.569824,5197.350098,5202.390137,5202.390137,3278180000


In [ ]:
# Connect to DuckDB
con = duckdb.connect(database=':memory:', read_only=False)
# Add a rowid column to the gspc_data table
gspc_data['rowid'] = gspc_data.reset_index().index

# Add a rowid column to the mxx_data table
mxx_data['rowid'] = mxx_data.reset_index().index
# Write the Pandas DataFrames to DuckDB
con.register('gspc_data', gspc_data)
con.register('mxx_data', mxx_data)

# Calculate the 5-year growth for each index
result = con.execute('''
    SELECT
      (SELECT "Adj Close" FROM gspc_data ORDER BY rowid LIMIT 1) AS gspc_start_price,
      (SELECT "Adj Close" FROM gspc_data ORDER BY rowid DESC LIMIT 1) AS gspc_end_price,
      (SELECT "Adj Close" FROM mxx_data ORDER BY rowid LIMIT 1) AS mxx_start_price,
      (SELECT "Adj Close" FROM mxx_data ORDER BY rowid DESC LIMIT 1) AS mxx_end_price
''')

# Fetch the result as a DataFrame
growth_df = result.fetchdf()

# Calculate growth rates
growth_df['gspc_growth'] = ((growth_df['gspc_end_price'] / growth_df['gspc_start_price']) - 1) * 100
growth_df['mxx_growth'] = ((growth_df['mxx_end_price'] / growth_df['mxx_start_price']) - 1) * 100

# Determine the higher growing index and its growth rate
if growth_df['gspc_growth'][0] > growth_df['mxx_growth'][0]:
    higher_growing_index = '^GSPC'
    higher_growth_rate = round(growth_df['gspc_growth'][0])
else:
    higher_growing_index = '^MXX'
    higher_growth_rate = round(growth_df['mxx_growth'][0])

# Print the results
print(f"Higher growing index: {higher_growing_index}")
print(f"Growth rate: {higher_growth_rate}%")

Higher growing index: ^GSPC
Growth rate: 81%


In [15]:
#Bonus
# Download historical data for SPY, gspc 50, and Nikkei 225

nikkei_data = pd.DataFrame()



nikkei_data = yf.download('^N225', start='2004-04-08', end='2024-04-10')

nikkei_data.reset_index(inplace=True)
con = duckdb.connect(database=':memory:', read_only=False)

[*********************100%%**********************]  1 of 1 completed


In [16]:
# Calculate the 10-year and 20-year growth for each index
result = con.execute('''
    SELECT
      (SELECT "Adj Close" FROM nikkei_data WHERE nikkei_data.Date = '2014-04-09') AS nikkei_start_price_10y,
      (SELECT "Adj Close" FROM nikkei_data WHERE nikkei_data.Date = '2024-04-09') AS nikkei_end_price_10y,
      (SELECT "Adj Close" FROM nikkei_data WHERE nikkei_data.Date = '2004-04-09') AS nikkei_start_price_20y,
      (SELECT "Adj Close" FROM nikkei_data WHERE nikkei_data.Date = '2024-04-09') AS nikkei_end_price_20y
''')

# Fetch the result as a DataFrame
growth_df = result.fetchdf()

# Calculate 10-year and 20-year growth rates

growth_df['nikkei_growth_10y'] = ((growth_df['nikkei_end_price_10y'] / growth_df['nikkei_start_price_10y']) - 1) * 100
growth_df['nikkei_growth_20y'] = ((growth_df['nikkei_end_price_20y'] / growth_df['nikkei_start_price_20y']) - 1) * 100

# Calculate the average yearly growth rate (CAGR) for each index

growth_df['nikkei_cagr_10y'] = ((1 + growth_df['nikkei_growth_10y'] / 100) ** (1 / 10) - 1) * 100
growth_df['nikkei_cagr_20y'] = ((1 + growth_df['nikkei_growth_20y'] / 100) ** (1 / 20) - 1) * 100

# Create a DataFrame with the growth rates and CAGR
data = {
    'Index': ['Nikkei 225'],
    '10-Year Growth (%)': [ growth_df['nikkei_growth_10y'][0]],
    '20-Year Growth (%)': [growth_df['nikkei_growth_20y'][0]],
    'Average Yearly Growth Rate (CAGR) for 10 Years (%)': [growth_df['nikkei_cagr_10y'][0]],
    'Average Yearly Growth Rate (CAGR) for 20 Years (%)': [growth_df['nikkei_cagr_20y'][0]]
}
table_df = pd.DataFrame(data)

# Display the table
print(table_df)

        Index  10-Year Growth (%)  20-Year Growth (%)  \
0  Nikkei 225           178.13979           234.29793   

   Average Yearly Growth Rate (CAGR) for 10 Years (%)  \
0                                           10.77106    

   Average Yearly Growth Rate (CAGR) for 20 Years (%)  
0                                           6.220095   


## Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks
Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023

Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value. Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

In [ ]:
# List of selected stocks
stocks = ["2222.SR", "BRK-B", "AAPL", "MSFT", "GOOG", "JPM"]

# Download historical data for each stock for the year 2023
data = {stock: yf.download(stock, start='2023-01-01', end='2023-12-31') for stock in stocks}

# Connect to DuckDB
con = duckdb.connect(database=':memory:', read_only=False)

# Write the Pandas DataFrames to DuckDB
for stock, df in data.items():
    df['symbol'] = stock  # Add the symbol column
    con.register(stock, df)

# Calculate the maximum and minimum "Adj Close" prices for each stock
result = con.execute('''
    SELECT
        symbol,
        MAX("Adj Close") AS max_adj_close,
        MIN("Adj Close") AS min_adj_close
    FROM
        (SELECT
            symbol,
            "Adj Close"
        FROM
            "2222.SR"
        UNION ALL
        SELECT
            symbol,
            "Adj Close"
        FROM
            "BRK-B"
        UNION ALL
        SELECT
            symbol,
            "Adj Close"
        FROM
            "AAPL"
        UNION ALL
        SELECT
            symbol,
            "Adj Close"
        FROM
            "MSFT"
        UNION ALL
        SELECT
            symbol,
            "Adj Close"
        FROM
            "GOOG"
        UNION ALL
        SELECT
            symbol,
            "Adj Close"
        FROM
            "JPM"
        )
    GROUP BY
        symbol
''')

# Fetch the result as a DataFrame
max_min_df = result.fetchdf()

# Calculate the range ratio for each stock
max_min_df['range_ratio'] = ((max_min_df['max_adj_close'] - max_min_df['min_adj_close']) / max_min_df['max_adj_close']).round(2)

# Find the stock with the largest range ratio
largest_range_stock = max_min_df.loc[max_min_df['range_ratio'].idxmax()]['symbol']
largest_range_ratio = max_min_df.loc[max_min_df['range_ratio'].idxmax()]['range_ratio']

# Print the results
print(f"Stock with the largest range ratio: {largest_range_stock}")
print(f"Largest range ratio: {largest_range_ratio}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Stock with the largest range ratio: MSFT
Largest range ratio: 0.42


##(Additional: why this may be important for your research?)
The range in stock trading is crucial for understanding volatility and identifying trading opportunities. A wider range indicates higher volatility, while a narrower range suggests stability. Support and resistance levels within a range help traders determine entry and exit points, aiding in informed decision-making for buying and selling.

## Question 5. [Stocks] Dividend Yield
Find the largest dividend yield for the same set of stocks

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023. You can use get_actions() method or .dividends field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximum value in % and round to 1 digit after the decimal point. (E.g., if you obtained 1.25 dollar dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)




In [ ]:
import yfinance as yf

# Define the list of company symbols
companies = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

# Dictionary to store the sum of dividends and last trading day Adj.Close price
dividends = {}
adj_close = {}

# Download dividends and Adj.Close prices
for company in companies:
    stock = yf.Ticker(company)
    dividends[company] = sum(stock.dividends.loc['2023'])
    adj_close[company] = stock.history(period='1d', start='2023-12-31')['Close'][0]

# Calculate dividend yield
dividend_yield = {company: (dividends[company] / adj_close[company]) * 100 for company in companies}

# Find the largest dividend yield
largest_dividend_yield = max(dividend_yield.values())

# Round to 1 digit after the decimal point
largest_dividend_yield_rounded = round(largest_dividend_yield, 1)

print(f"Largest dividend yield: {largest_dividend_yield_rounded}%")

Largest dividend yield: 2.8%


## Question 6. [Exploratory] Investigate new metrics

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).

**Volatilty Metric**
1.  Indicators:Beta,Beta Volatility,Maximum Drawdown (MDD),14-Day Average True Range (ATR) ,
2.  Time series:  Bollinger Bands to analyze standard deviation over time

**Momentum  Data**
1.  Indicators:14-Day Relative Strength Index (RSI),Moving Average Convergence Divergence (MACD),Rate of Change, Stochastic Oscillator

**Volume Metric**
1.  Indicators:On-Balance Volume (OBV),Klinger Oscillator
2.  Time series: Chaikin Money Flow

**Company Fundamental Data**
1.  Indicators:PE Ratio,PB Ratio, Dividend Yield

2.  Time series: PE Ratio (10y Range) ,PB Ratio (10y Median),Dividend Yield % (10y Range)





## Question 7. [Exploratory] Time-driven strategy description around earnings releases
Free text answer

Explore earning dates for the whole month of April - e.g. using YahooFinance earnings calendar (https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23). Compare with the previous closed earnings (e.g., recent dates with full data https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08).

Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.

I would imagine that we will need to compare and contrast the current earning with the previous earning.

1.  Income statement (a) Revenue, (b) expenses, (c) gains, and (d) losses
2.  Balance Sheet (a) Assest long/short term, (b) Liability long/short term, (c) Shareholder equity
3.  Cash Flow analysis (a) Cash flow from operating activities
(b) Cash flow from investing activities, (c) Cash flow from financing activities, (d)Disclosure of non-cash activities
4.  Company Market risk

https://www.investopedia.com/terms/i/incomestatement.asp

https://www.investopedia.com/investing/what-is-a-cash-flow-statement/

https://www.investopedia.com/terms/f/financial-statements.asp

https://www.investopedia.com/terms/m/marketrisk.asp
